In [1]:
def tool():
    '''
    read_txt ,get_maintext ,get_content ,read_json ,search_json ,json_container ,add_table ,one_accused ,one_accused_txt ,
    digit_maker ,drop_digit_para ,re_find_digits ,get_digits ,re_trade_para ,re_sell_drug2 ,
    '''
    pass   

def read_txt(file,num = 1):
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    if num == 1:
        return txt.replace('\n\n','\n')
    elif num == 2:
        return txt.replace('\n','').replace('\u3000','').replace(' ','').replace(',','')
    elif num ==3:
        return '書記官'.join(txt2.split('書記官')[:-1])
    elif num ==4: #處理有table分割#格式的
        return txt.replace('\n\n','\n').replace('\u3000','').replace(' ','').replace(',','')

def get_maintext(file,_type=None):
    ''' 輸出主文內容 '''
    if not _type:
        print('Error- miss 1 argument: maintent(file,_type= \'path\' or \'string\')')
        return
    if _type=='path':
        with open(file,encoding='utf-8-sig') as f:
            txt = f.read()
    else:  # 'string'
        txt = file
    txt = txt.replace(' ','').replace('\u3000','').replace('\n\n','\n').replace('\r','')
    if '\n主文\n' in txt:
        part = '\n主文\n'.join(txt.split('\n主文\n')[1:])
    else:
        return txt
    regex = '\\n\w{1,9}[^。]\\n' #[\u4e00-\u9fa5]中文字
    end = re.search(regex,part)
    if end:
        end_str = end[0]
        #print(end_str)
        maintext = part.split(end_str)[0]
        return maintext
    else:
        return '書記官'.join( part.split('書記官')[:-1])

def get_content(file,_type=None):
    ''' 輸出內文內容 '''
    if  _type !='path' and _type != 'string':
        print('Error- miss 1 argument: maintent(file,_type= \'path\' or \'string\')')
        return
    elif _type=='path':
        with open(file,encoding='utf-8-sig') as f:
            txt = f.read()
    elif _type =='string':
        txt = file
    txt = txt.replace(' ','').replace('\u3000','').replace('\n\n','\n').replace('\r','')
    txt2 = '書記官'.join( txt.split('書記官')[:-1])
    if '\n主文\n' in txt2:
        part = '\n主文\n'.join(txt2.split('\n主文\n')[1:])
    else:
        part = txt2
    #print(part)
    regex = '\\n\w{1,9}[^。]\\n' #[\u4e00-\u9fa5]中文字
    start = re.search(regex,part)
    #print(start)
    if start :
        start_str = start[0]
        #print(start_str)
        content = part.split(start_str)[1].replace('\n','')
        return content
    else:
        return part
    
def read_json(file):
    with open(file,encoding= 'utf-8-sig') as f:
        f_json = json.load(f)
    cause = f_json['JTITLE'] #案由
    full = f_json['JFULL'] #全文
    main = get_maintext(full,'string') #主文
    content = get_content(full,'string') #內文
    return cause,full,main,content

def search_json(file,search_cause=None,search_main=None,search_content=None,model='part'):
    """
    search_json(file,search_cause=None,search_main=None,search_content=None,model='part' or 'full')
    full:完全匹配
    part:至少一個匹配
    符合條件函數回傳file，不符合則回傳None
    """
    cause,full,main,content = read_json(file)
    input_num = 0
    match_num = 0
    onoff = False
    if search_cause:
        input_num = input_num + 1
        if search_cause == cause: #案由需要完全輸入正確
            match_num = match_num + 1
            onoff = True
    if search_main:
        input_num = input_num + 1
        if search_main in main:
            match_num = match_num + 1
            onoff = True
    if search_content:
        input_num = input_num + 1
        if search_content in content:
            match_num = match_num + 1
            onoff = True
    if model == 'full':
        if match_num == input_num and onoff:
            #print(match_num)
            return file
        else:
            return
    if onoff:
        #print(match_num,input_num)
        return file
    else:
        return

def json_container(path_list,search_cause=None,search_main=None,search_content=None,model='part'):
    if type(path_list) != list:
        print('first argument require list')
        return
    target=[]
    for it,file in enumerate(files):
        if it%1000 ==0:
            print(it,end=',')
        path = search_json(file ,search_cause ,search_main ,search_content ,model )
        if path: #path不等於None的話就存進target
            target.append(path)
    return target    

def add_table(file):
    with open(file,encoding='utf-8-sig') as f:
        txt = f.read()
    if '\ntable\n' in txt:
        print('already add table')
        return
    txt1 = txt.replace('\n\n','\n')
    #txt2 = txt.replace(' ','').replace('\u3000','').replace('\n','')#.replace('\n\n','\n')
    #txt2 = '\n附錄'.join(txt1.split('\n附錄')[1:])
    if '┌' not in txt1: #無表格
        return ''
    tables = txt1.split('┌') #一個判決書中多個附表
    all_str = '' #存全部表格內的文字
    for n1,table in enumerate(tables): #每一個附表
        if n1 == 0: #上面的內文
            continue
        table = table.replace(' ','').replace('\u3000','').replace('\n\n','\n')
        table_str = '' 
        cases = table.split('\n├') #一個附表中多個編號(紀錄)
        #定位出有欄位名稱的case
        onoff = '無編號'
        for n2,case in enumerate(cases):
            if '\n│編' in case: #有編號這兩字在上下兩行的
                n_col = n2
                onoff = '有編號'
                break
        if onoff == '無編號':
            continue
        # 建立欄位名/
        column = []
        cases[n_col] = cases[n_col].replace('├','│').replace('┤','│')
        for n3,row in enumerate(cases[n_col].splitlines()): #欄位名稱整合
            if len(row.split('│')) == 3 or row == '': # 只有一個格子，ex: '│四、本院...│' 需在此迴圈下新寫一個紀錄方式
                #print(n3,'跳過')
                continue
            if '│' == row[0] and column != []: #'內容再加'
                for j,cel in enumerate(row.split('│')[1:-1]):
                    if j == leng: #追加新一欄
                        column.append(cel)
                        leng = leng +1 #目前欄位長度+1
                    else:
                        column[j] = column[j] + cel
                continue
            elif '│' == row[0] and '┬' not in row: #'創立'
                #print(n3,'創立')
                for index in row.split('│')[1:-1]:
                    column.append(index)
                leng = len(column)
        for p in range(len(column)):
            column[p] = column[p] + ':'
        #將欄位內容整合
        for k,case in enumerate(cases):#從1開始
            if k <= n_col:
                continue
            case_str = ''
            c = column.copy()
            case = case.replace('├','│').replace('┤','│')
            for r in case.splitlines():
                if r == '':
                    continue
                if '│' != r[0]:
                    continue
                temp = r.split('│')[1:-1] #最前跟後是''
                for j,cell in enumerate(temp):
                    if j >= len(c): #因無故多了一個分隔直線
                        break
                    c[j] = c[j] + cell
            case_str = '#' +'\n#'.join(c)
            table_str = table_str + '##' + case_str + '\n' 
        all_str = all_str + '#' + str(n1) + '#\n' + table_str #每個tabel的第一行為#i# #改這裡
    if all_str == '':
        return ''
    all_str = '\ntable\n'+ all_str
    return all_str

def one_accused(file):
    '''單被告的名字'''
    with open(file,encoding='utf-8-sig') as f:
        txt =f.read()
    name = one_accused_txt(txt)
    return name

def one_accused_txt(txt):
    txt2 = txt.replace('\n\n','\n').replace('\u3000','').replace(' ','')
    txt3 = txt2.split('主文')[0]
    txt3 = txt3.replace('(','（').replace(')','）')
    text = txt3
    regex = "（.*）"
    pattern = re.compile(regex)
    res = re.findall(pattern,text)
    for word in res:
        txt3 = txt3.replace(word,'')
    lines = txt3.splitlines()
    name = '被告'
    for l in lines:
        if '被告' == l[:2] and '，' not in l and len(l) > 3:
            name = l[2:]
            break
        elif '即被告' == l[:3] and '即被告之' != l[:4] and len(l) > 3:
            name = l[3:]
            break
        elif '上訴人' == l[:3] and '檢察官' not in l and len(l) > 3:
            name = l[3:]
            break
    cut = ['原名','（','民國','【','男','女','1','2','3','4','5','6','7','8','9']
    if len(name) > 3:
        if '即被告' in name:
            name = name.split('即被告')[1]
        for c in cut:
            if c in name:
                name = name.split(c)[0]
    if name == '':
        name = '被告'
    return name

def _capital_chars():
    capital_chars = {
        u'零': 0,u'壹': 1,u'貳': 2,u'參': 3,u'': 3, # \ue372
        u'': 3, # \ue373
        u'肆': 4,u'伍': 5,u'陸': 6,u'柒': 7,u'捌': 8,u'玖': 9,
        u'一': 1,u'二': 2,u'三': 3,u'四': 4,u'五': 5,u'六': 6,u'七': 7,u'八': 8,u'九': 9,
        '0': 0,u'1': 1,u'2': 2,u'3': 3,u'4': 4,u'5': 5,u'6': 6,u'7': 7,u'8': 8,u'9': 9,
        '０': 0,'１': 1,'２': 2,'３': 3,'４': 4,'５': 5,'６': 6,'７': 7,'８': 8,'９': 9
    }
    return capital_chars
def _units():
    units = {
        u'垓': 10 ** 20,
        u'京': 10 ** 16,
        u'兆': 10 ** 12,
        u'億': 10 ** 8,
        u'萬': 10 ** 4,
    }
    return units
def _turn_digits(source,end = '元'):
    # reference: https://github.com/nt90561/get_attribute/blob/master/convert_digit.py
    # Convert Chinese uppercase numbers to digits
    # 注意有級距的可能會出錯 ex: 15至20萬
    capital_chars = _capital_chars()
    digits = { c: d for c, d in capital_chars.items() }
    units = _units()
    single_units = {
        u'仟': 1000,
        u'千': 1000,
        u'佰': 100,
        u'百': 100,
        u'拾': 10,
        u'十': 10,
    }
    # 先宣告所有需要用到的緩衝區
    digit, buf, result = 0, 0, 0
    # 列舉每一個字元
    for char in source:
        # 如果遇到大寫數字，先存起來
        if char in capital_chars:
            digit = capital_chars[char]
        # 如果遇到可重複的單位，存進緩衝區
        elif char in single_units:
            if digit == 0 and (char == u'拾' or char == u'十'):
                digit = 1
            buf += digit * single_units[char]
            digit = 0
        # 不會重複的單位可以直接結算
        else:
            buf += digit
            digit = 0
            if char in units:
                result += buf * units[char]
                buf = 0
            elif char == end :
                break
    if source == u"拾萬"+end or source == u'十萬'+end:
        result = 100000
    # 結算所有緩衝區的數字
    total = result + buf + digit
    if not total:
        return source
    else:
        return int(total)


def digit_maker(i,end= '元'): #段落i中第一個數值
    #use: _turn_digits,_units(),_capital_chars(),
    units = _units()
    capital_chars = _capital_chars()
    all_unit=['十','拾','百','佰','千','仟','萬','億','兆'] 
    turn_mix = {'十':'0','拾':'0','百':'00','佰':'00','千':'000','仟':'000','萬':'0'*4,'億':'0'*8,'兆':'0'*12}
    onoff2 = None
    if '.' in i and end != '元': #有小數
        nums = i.split('.')[-2:] # '1.3公克' [ 1, 3公克 ]
        interger = nums[0]
        decimal = nums[1]
        onoff2 = '有小數'
    else:
        interger = i
    #print(interger)
    onoff = '數字表達'
    buff = None
    for u in units: #for 53萬元
        if u+end == interger[-2:]:
            interger = interger[:-2] + end #拿掉unit
            buff = str(units[u])[1:] #{'萬': 10000 }
            #print(i,buff) #i=53 ,buff = 0000
    for unit in all_unit: 
        if unit in interger: #有中文單位在裡面
            if re.search('[1-9][1-9]|[1-9]0[1-9]',i) != None:
                for ca in capital_chars:
                    interger = interger.replace(ca,str(capital_chars[ca]) )
                #print(interger,end=',')
                reg ='(\d+[\u4e00-\u9fa5]{1,2})'
                for times in range(10,-1,-1):
                    pattern= reg*times+'(\d+[\u4e00-\u9fa5]{0,3})元'
                    result2 = re.search(pattern,interger)
                    if result2 != None:
                        #print(pattern,result2)
                        temp = 0
                        for piece in result2.groups():
                            #print('piece:',piece)
                            pie = piece
                            for u in turn_mix:
                                pie = pie.replace(u,turn_mix[u])
                            #print(pie,end=',')
                            pie2 = re.search('[^0]\d+',pie).group()
                            #print(pie2)
                            temp = temp + int(pie2)
                        #print('t:',temp)
                        onoff = '混合表達'
                        break
            else:
                turn_0 = {'000':'千','00':'百','0':'十'}
                for con in turn_0.keys(): #for: 五千400元
                    interger = interger.replace(con,turn_0[con])
                temp = _turn_digits(interger,end) #正規大寫數值
                onoff = '中文字'
                break
    if onoff == '數字表達': #for: 123,456元
        for cap in capital_chars: #for: 肆伍陸元
            interger= interger.replace(cap,str( capital_chars[cap]))
        if re.findall('\d+',interger) != []:
            temp = re.findall('\d+',interger)[0]
    if type(buff) == str:
        try:
            temp = str(temp) + buff
        except UnboundLocalError:
            pass
    if onoff2 == '有小數':
        for cap in capital_chars: #for: 肆伍陸元
            decimal= decimal.replace(cap,str( capital_chars[cap]))
        if re.findall('\d+',decimal) != []:
            dec = re.findall('\d+',decimal)[0]
        try:
            temp = str(temp) + '.' + dec
        except UnboundLocalError:
            pass
    try:
        temp = str(temp)
        if re.search('[1-9]+',temp) == None:
            return 
        elif len(temp)>=2 and temp[0] =='0' and temp[1] != '.':
            temp = '0.' + temp[1:]
    except UnboundLocalError:
        return 
    return temp

def drop_digit_para(source,end='元',
                    drops=['扣案','沒收','誤載','賒欠','差額','原審','餘款','賺','遊戲幣','再收取','費用',
                           '愷他命','K他命','海洛因','驗餘'
                          ]): #移除文章內有drops及數值的句子
    condition = [
        '[\w（）\(\),、\<\>\[\]\-\.「」:：％]\w*誤載\w*[\w（）\(\),、\<\>\[\]\-\.「」:：％]',
        '\w*{drop}\w*[\w（）\(\),、\<\>\[\]\-\.「」:：％]\w*{end}\w*[\w（）\(\),、\<\>\[\]\-\.「」:：％]\w*',            
        '\w*{drop}\w*[\w（）\(\),、\<\>\[\]\-\.「」:：％]\w*[\w（）\(\),、\<\>\[\]\-\.「」:：％]\w*{end}\w*',
        '\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{end}\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{drop}\w*',
        '\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{end}\w*{drop}\w*',
        '\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{end}\w*{drop}\w*',
        '\w*{end}\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{drop}\w*',
        '\w*{drop}\w*[\w（）\(\),、\<\>\[\]\-\.「」\:\：％]\w*{end}\w*',
        '\w*{drop}\w*{end}\w*',
        '\w*{end}\w*{drop}\w*',
        '\(\w*賺\w*元\)',
            ]
    for drop in drops:
        regex = '|'.join(condition).format(drop=drop,end=end)
        paras = re.findall(regex,source)
        #print(drop,paras,sep=' ------- ')
        if len(paras) == 0:
            #print('not match:',drop)
            continue
        for p in paras: #拿掉drop關鍵詞篩出的句子
            source = source.split(p)[0] + ''.join(source.split(p)[1:])
            #print(p,source,sep=' ------- ')
    return source

def re_find_digits(source,end='元'):
    regex = '[\d零壹貳參肆伍陸柒捌玖一二三四五六七八九十拾百佰千仟萬億兆,.點]*{}'.format(end)
    digits = re.findall(regex,source)
    return digits

def get_digits(source,end = '元',onoff= False,
               drops=['扣案','沒收','誤載','賒欠','差額','原審','餘款','遊戲幣','再收取','費用','愷他命','K他命','海洛因','驗餘'
                                                  ]): #有級距時取高價的 
    if type(source) != str:
        print('input string')
        return
    #愷他命、海洛因 斟酌加進drop
    #use: digit_maker(i),drop_digit_para,re_find_digits
    if onoff:
        source = drop_digit_para(source,end,drops=drops)
    #print(source)
    digits = re_find_digits(source,end)
    #print(digits)
    values = []
    for i in digits:    
        i = i[-15:].replace(',','').replace('點','.').replace('‧','.')
        #print(i)
        temp = digit_maker(i,end)
        #print(temp)
        if temp != None:
            values.append(temp)
    values = [str(x) for x in values]
    return values

def re_trade_para(txt,name):
    condition = ['.{1,20}[^、之以]販[賣|售].{1,40}元.{1,30}公克.{1,15}',
                 '.{1,20}[^、之以]販[賣|售].{1,20}公克.{1,40}元.{1,15}',
                 '.{1,20}元.{1,40}[^、之以]販[賣|售].{1,20}公克.{1,15}',
                 '.{1,20}元.{1,40}公克.{1,40}[^、之以]販[賣|售].{1,15}',
                 '.{1,20}公克.{1,40}元.{1,40}[^、之以]販[賣|售].{1,15}',
                 '.{1,20}公克.{1,40}[^、之以]販[賣|售].{1,20}元.{1,15}',
                 '向[被|甲][告|○].{1,15}買.{1,30}公克.{1,30}元.{1,15}',
                 '向'+name+'.{1,15}買.{1,30}公克.{1,30}元.{1,15}',
                 '向[被|甲][告|○].{1,15}買.{1,30}元.{1,30}公克.{1,15}',
                 '向'+name+'.{1,15}買.{1,30}元.{1,30}公克.{1,15}',
                 '向[被|甲][告|○].{1,20}元.{1,15}買.{1,30}公克.{1,15}',
                 '向'+name+'.{1,20}元.{1,15}買.{1,30}公克.{1,15}',
                 '向[被|甲][告|○].{1,20}元.{1,30}公克.{1,15}買.{1,15}',
                 '向'+name+'.{1,20}元.{1,30}公克.{1,15}買.{1,15}',
                 '.{1,10}元.{1,8}向[被|甲][告|○].{1,20}公克.{1,15}買.{1,15}',
                 '.{1,10}元.{1,8}向'+name+'.{1,20}公克.{1,15}買.{1,15}',
                 '.{1,10}元.{1,8}向[被|甲][告|○].{1,15}買.{1,30}公克.{1,15}',
                 '.{1,10}元.{1,8}向'+name+'.{1,15}買.{1,30}公克.{1,15}',
                 '.{1,10}元.{1,30}公克.{1,8}向[被|甲][告|○].{1,15}買.{1,15}',
                 '.{1,10}元.{1,30}公克.{1,8}向'+name+'.{1,15}買.{1,15}',
                 '.{1,20}[^、之以]販[賣|售].{1,40}元',
                 '.{1,20}元.{1,30}[^、之以]販[賣|售]',
                 '向[被|甲][告|○].{1,10}買.{1,30}元.{1,20}',
                 '向'+name+'.{1,15}買.{1,30}元.{1,20}',
                 '.{1,10}元.{1,8}向[被|甲][告|○].{1,15}買.{0,20}',
                 '.{1,20}[^、之以]販[賣|售].{1,40}公克',
                 '.{1,20}公克.{1,30}[^、之以]販[賣|售]',
                 '向[被|甲][告|○].{1,10}買.{1,30}公克.{1,20}',
                 '向'+name+'.{1,15}買.{1,30}公克.{1,20}',
                 '.{1,10}公克.{1,8}向[被|甲][告|○].{1,15}買.{0,20}',
                 
                ]
    regex = '|'.join(condition)
    result = re.findall(regex,txt)
    return result

def re_sell_drug2(txt,name):
    con = [
        '被告[又]{0,1}.{,30}[共同]{0,2}販賣[第]{0,1}二級毒品',
        '甲○○[又]{0,1}.{,30}[共同]{0,2}販賣[第]{0,1}二級毒品',
        name+'[又]{0,1}.{,30}[共同]{0,2}販賣[第]{0,1}二級毒品',
        '基於販賣.{0,20}第二級毒品',
        '被告[又]{0,1}[共同]{0,2}販賣.{1,10}、[第]{0,1}二級毒品',
        '甲○○[又]{0,1}[共同]{0,2}販賣.{1,10}、[第]{0,1}二級毒品',
        name+'[又]{0,1}[共同]{0,2}販賣.{1,10}、[第]{0,1}二級毒品',
            ]
    regex = '|'.join(con)
    result = re.search(regex,txt)
    return result
    